In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('../input/sartorius-cell-instance-segmentation/train.csv')
df1 = df.groupby('id').agg(list).reset_index()

In [ ]:
df1[df1.id=='0030fd0e6378'].cell_type 

In [ ]:
#df[df.id=='ffdb3cc02eef'] 

In [ ]:
 
for col in df.columns[2:]:
    df1[col] = df1[col].apply(
        lambda x: np.unique(x)[0] if len(np.unique(x)) == 1 else np.unique(x)
    )

In [ ]:
def rles_to_mask(encs, shape):
    """
    Decodes a rle.
    #Thanks to Theovel
    Args:
        encs (list of str): Rles for each class.
        shape (tuple [2]): Mask size.

    Returns:
        np array [shape]: Mask.
    """
    img = np.zeros(shape[0] * shape[1], dtype=np.uint)
    mask_sum=np.zeros(len(encs))
    for m, enc in enumerate(encs):
         
        if isinstance(enc, np.float) and np.isnan(enc):
            continue
        enc_split = enc.split()
        for i in range(len(enc_split) // 2):
            start = int(enc_split[2 * i]) - 1
            length = int(enc_split[2 * i + 1])
            img[start: start + length] = 1 + m
        ##index=np.where(img==1+m)
            
        #print(img.sum())     
        mask_sum[m]=img[np.where(img==1+m)].sum()
        #break
    return img.reshape(shape),mask_sum

In [ ]:
import cv2

In [ ]:
#i=cv2.imread('../input/sartorius-cell-instance-segmentation/train/0728b8f39241.png')
#i=cv2.  equalizeHist(i.astype('int'))

In [ ]:
from matplotlib import pyplot as plt
#plt.imshow(i)

In [ ]:
#df1['annotation']

In [ ]:
mask_class_dict={}
df1['mask_sum']=0

for i in range(df1.shape[0]):
    

    shape = df1[['height', 'width']].values[i]

    mask=rles_to_mask(df1['annotation'][i],shape)
    df1.loc[i,'mask_sum']=np.ceil(np.mean(mask[1] ))

In [ ]:
pd.set_option("display.max_rows", 500)

In [ ]:
#class_dict={'shsy5y':10,'astro':20,'cort':30}
#df1['class_mask_sum']=df1.cell_type.map(class_dict)
plt.hist(df1.mask_sum,30)

In [ ]:
!  pip install category_encoders

In [ ]:
#df1[df1.cell_type.str.contains('ast')].mask_sum.hist()

#df1['class_mask_sum_int']=df1.apply(lambda x: int((x['mask_sum'])%5),axis=1)
import category_encoders as ce
from sklearn import preprocessing
df1['mask_sum_quantiles'] = pd.qcut(df1['mask_sum'], q=25, precision=0)

#df1.mask_sum_quantiles.value_counts() 

df1.groupby(['mask_sum_quantiles','cell_type']).id.count()
le = preprocessing.LabelEncoder()

df1['mask_class']=le.fit_transform(df1['mask_sum_quantiles'])

In [ ]:
df1.groupby(['mask_class','cell_type']).id.count()

In [ ]:
df1.cell_type.value_counts()

In [ ]:
from sklearn.model_selection import StratifiedKFold

skf=StratifiedKFold(n_splits=5, random_state=42, shuffle=True) 
df1['fold']=-1
fold=0
#for trn_id,val_id in skf.split(df1.id.values,df1.class_mask_sum_int):
for trn_id,val_id in skf.split(df1.id.values,df1.mask_class):
    #print(val_id)
    df1.loc[val_id,'fold']=fold
    fold=fold+1
    


In [ ]:
df1.groupby(['fold','cell_type','mask_class']).id.count()

In [ ]:
df1.groupby(['fold','cell_type' ]).id.count()

In [ ]:
#mask.shape

#mask[0][np.where(mask[0]==1)].sum()

df1.to_csv('sartorius_stratified_folds.csv',index=False)

In [ ]:
#df1['class_length']=df1.cell_type.apply(lambda x :  np.array(x).shape[0])

#Please let me know how does it correlates to LB
 

In [ ]:
#df1.class_length.unique()